# Basic Test

In [ ]:
from Go2Py.sim.mujoco import Go2Sim
import numpy as np

In [ ]:
robot = Go2Sim()
robot.standUpReset()

In [ ]:
from Go2Py.control.walk_these_ways import *

In [ ]:
checkpoint_path = "/home/rstaion/projects/rooholla/locomotion/Go2Py/Go2Py/assets/checkpoints/walk_these_ways/"

cfg = loadParameters(checkpoint_path)
policy = Policy(checkpoint_path)
command_profile = CommandInterface()

In [ ]:
agent = WalkTheseWaysAgent(cfg, command_profile, robot)
agent = HistoryWrapper(agent)

In [ ]:
control_dt = cfg["control"]["decimation"] * cfg["sim"]["dt"]
simulation_dt = robot.dt
obs = agent.reset()

In [ ]:
robot.reset()
obs = agent.reset()
for i in range(5000):
    policy_info = {}
    action = policy(obs, policy_info)
    if i % (control_dt // simulation_dt) == 0:
        obs, ret, done, info = agent.step(action)
    robot.step()
    command_profile.yaw_vel_cmd = 0.0
    command_profile.x_vel_cmd = 0.0
    command_profile.y_vel_cmd = 0.0
    command_profile.stance_width_cmd=0.2
    command_profile.footswing_height_cmd=-0.05
    command_profile.step_frequency_cmd = 2.5
    time.sleep(robot.dt/4)

# Full System With FSM

In [1]:
from Go2Py.robot.fsm import FSM
from Go2Py.robot.remote import KeyboardRemote
from Go2Py.robot.safety import SafetyHypervisor
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.control.walk_these_ways import *
import numpy as np

In [2]:
class walkTheseWaysController:
    def __init__(self, robot, remote, checkpoint):
        self.remote = remote
        self.robot = robot
        self.cfg = loadParameters(checkpoint)
        self.policy = Policy(checkpoint)
        self.command_profile = CommandInterface()
        self.agent = WalkTheseWaysAgent(self.cfg, self.command_profile, self.robot)
        self.agent = HistoryWrapper(self.agent)
        self.init()

    def init(self):
        self.obs = self.agent.reset()
        self.policy_info = {}
        self.command_profile.yaw_vel_cmd = 0.0
        self.command_profile.x_vel_cmd = 0.0
        self.command_profile.y_vel_cmd = 0.0
        self.command_profile.stance_width_cmd=0.2
        self.command_profile.footswing_height_cmd=-0.05
        self.command_profile.step_frequency_cmd = 2.5
        self.command_profile.bodyHeight = 0.05

    def update(self, robot, remote):
        action = self.policy(self.obs, self.policy_info)
        self.obs, self.ret, self.done, self.info = self.agent.step(action)


In [3]:
remote = KeyboardRemote()
robot = Go2Sim()
robot.standUpReset()
safety_hypervisor = SafetyHypervisor(robot)

In [4]:
checkpoint = "/home/rstaion/projects/rooholla/locomotion/Go2Py/Go2Py/assets/checkpoints/walk_these_ways/"
controller = walkTheseWaysController(robot, remote, checkpoint)
fsm = FSM(robot, remote, safety_hypervisor, user_controller_callback=controller.update)

p_gains: [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


In [ ]:
robot.standUpReset()
controller.command_profile.roll_cmd=0.3

: 